В этой тетрадке я только добавила комментарии к коду из семинара. 

### natasha

Установка natasha ничего особенного не требует, просто выполните команду pip install natasha в командной строке / терминале. Наташа - библиотека NLP-инструментов только для русского языка, она очень быстрая, потому что многие вещи в ней работают на правилах. У Наташи есть несколько отдельных подбиблиотек, мы с вами уже знаем razdel. 

В отличие от spacy и других подобных библиотек-комбайнов, Наташа составная: вам придется вызывать по отдельности те процессоры для текста (морфологический, синтаксический...), которые вам нужны. Если не вызовете тот, без которого не работает нужный вам, не сработает тот, что вам нужен. 

In [ ]:
!pip install natasha

Морфосинтаксический парсинг

Мы должны импортировать из Наташи несколько классов: 

- Segmenter - для сегментации (токенизация и деление на предложения): без этого процессора, скорее всего, ни один другой нормально не отработает
- NewsEmbedding - эмбеддинги слов, то есть, векторные представления слов. Без эмбеддингов не работают морфо- и синтаксический парсеры, NER-тэггер. 
- NewsMorphTagger - морфопарсер
- NewsSyntaxParser - синтаксический парсер
- Doc - класс для хранения и обработки нашего текста

Также потом будем импортировать еще эти:

- NewsNERTagger - разметчик именованных сущностей
- MorphVocab - достает леммы из морфоразбора (без него не работает)

In [2]:
from natasha import (
    Segmenter,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    
    Doc
)

In [ ]:
segmenter = Segmenter()  # токенизация и разделение на предложения
emb = NewsEmbedding()  # эмбеддинги
morph_tagger = NewsMorphTagger(emb)  # морфология. Мы должны передать в морфопарсер эмбеддинги, потому что без них он не работает
syntax_parser = NewsSyntaxParser(emb) # синтаксис

text = '29 марта 2017 года правительство Великобритании инициировало процедуру выхода в соответствии со статьёй 50 Договора о Европейском союзе; первоначально планировалось, что Великобритания покинет Европейский союз через два года, 29 марта 2019 года в 23:00 по Гринвичу.'
doc = Doc(text)

doc.segment(segmenter)  # сегментируем наш текст
doc.tag_morph(morph_tagger)  # морфопарсим его
doc.parse_syntax(syntax_parser)  # и синтаксис разбираем 
sent = doc.sents[0]  # в атрибуте doc.sents хранится итератор со всеми нашими предложениями; под индексом 0 - первое предложение
sent.morph.print()  # внимательно: здесь print не функция, а метод для специализированного наташиного красивого вывода
sent.syntax.print()

Распознание именованных сущностей

In [ ]:
from natasha import NewsNERTagger

ner_tagger = NewsNERTagger(emb)
doc.tag_ner(ner_tagger)
doc.ner.print()

Лемматизация

In [ ]:
from natasha import MorphVocab

morph_vocab = MorphVocab()

for token in doc.tokens:
  token.lemmatize(morph_vocab)
  print(token.lemma)

Нормализация именованных сущностей

In [ ]:
for span in doc.spans:
    span.normalize(morph_vocab)
   
{_.text: _.normal for _ in doc.spans}

Упражнения

- Возьмите любой достаточно длинный новостной текст, извлеките из него все именованные сущности и нормализуйте их. 

In [ ]:
# your code here

- Постройте дерево зависимостей для одного и того же предложения в natasha и spacy или corpy (придется вручную записать файл .conll и отправить его содержимое в арборатор, например: как это сделать, уточните в лекции 6 за прошлый семестр). Сравните, что вам больше нравится. 

In [ ]:
# your code here

### Stanza

Эту библиотеку разрабатывают ученые из Стэнфордского университета (первоначально она называлась StanfordNLP и была написана в Java). Она тоже довольно популярная и, как и spacy, по сути является только общей оболочкой для разных нейронных моделек, к которым просто предоставляет интерфейс. Сами модельки, если вы заметили, скачиваются с известного сайта huggingface.co, где выкладываются обученные нейронные сети и датасеты. 

Станца похожа на spacy и для морфосинтаксиса включает в себя те же модели UDPipe. 

При установке может возникнуть проблема, если у вас уже был случайно установлен модуль pytorch не той версии; проверьте с помощью команды pip list, есть ли у вас pytorch 1.16 в списке, если нет - то все хорошо и можно устанавливать станцу, а если есть, то стоит его удалить pip uninstall pytorch. Если есть torch 1.12, то это ок. 

In [ ]:
!pip install stanza

Загрузка моделей

Модели станца загружает автоматически, когда вы создаете объект класса Pipeline. Обязательный аргумент для создания такого класса - только аббревиатура вашего языка (список доступных моделек есть в [документации станцы](https://stanfordnlp.github.io/stanza/available_models.html)). Необязательный атрибут - processors, куда можно в строке списком передать все процессоры, которые вы хотите включить, то есть, те модели, которые вам нужны. Например, в коде ниже я для русского языка включила все имеющиеся в наличии, потому что ничего не указала, для английского включила только токенизацию, морфопарсинг и парсинг составляющих, а для французского - токенизацию и multiword tokenization.

In [9]:
import stanza

In [ ]:
nlp_ru = stanza.Pipeline(lang='ru')
nlp_en = stanza.Pipeline(lang='en', processors='tokenize, pos, constituency')
nlp_fr = stanza.Pipeline(lang='fr', processors='tokenize, mwt')

Токенизация, сегментация по предложениям

Все остальное работает ровно так же, как в spacy. 

In [21]:
text = '''Архитектура Византии — совокупность традиций строительства и архитектуры в поздней Римской империи и в Византии в период с начала IV века по середину XV века. В качестве отдельных направлений исследования выделяют религиозную архитектуру Византии, византийскую фортификацию и гражданское строительство, включающее дворцы, общественные сооружения и частные дома. Также в рамках данной дисциплины изучают традиции строительного ремесла и декоративного искусства.'''

doc = nlp_ru(text)

In [17]:
print(*[sentence.text for sentence in doc.sentences], sep='\n')

Архитектура Византии — совокупность традиций строительства и архитектуры в поздней Римской империи и в Византии в период с начала IV века по середину XV века.
В качестве отдельных направлений исследования выделяют религиозную архитектуру Византии, византийскую фортификацию и гражданское строительство, включающее дворцы, общественные сооружения и частные дома.
Также в рамках данной дисциплины изучают традиции строительного ремесла и декоративного искусства.


Дальше у меня немного сложноватые генераторные выражения, но любое из этих выражений можно на самом деле развернуть в цикл. 

In [ ]:
for i, sentence in enumerate(doc.sentences):
  print(f'====== Sentence {i+1} tokens =======')
  print(*[f'id: {token.id}\ttext: {token.text}' for token in sentence.tokens], sep='\n')

Мультисловная токенизация - это когда, если у нас в языке есть сливающиеся слова, как во французском, токенизатор вернет слова, которые слились:

    du = de + le
    итал. del = de + il
    нем. im = in + dem
    ...

In [ ]:
text_fr = '''Il est révélé par les romans Extension du domaine de la lutte (1994) et, surtout, Les Particules élémentaires (1998), qui le fait connaître d'un large public.'''

doc_fr = nlp_fr(text_fr)
for token in doc_fr.sentences[0].tokens:
    print(f'token: {token.text}\twords: {", ".join([word.text for word in token.words])}')

Все следующие вещи на самом деле можно объединить в один большой цикл, который будет выводить всю информацию вообще: все, что есть в формате UD, содержится в атрибутах токенов. 

Лемматизация

In [ ]:
print(*[f'word: {word.text+" "}\tlemma: {word.lemma}' for sent in doc.sentences for word in sent.words], sep='\n')

Морфопарсинг

In [ ]:
print(*[f'word: {word.text}\tupos: {word.upos}\txpos: {word.xpos}\tfeats: {word.feats if word.feats else "_"}' for sent in doc.sentences for word in sent.words], sep='\n')

Парсинг синтаксических зависимостей

In [ ]:
print(*[f'id: {word.id}\tword: {word.text}\thead id: {word.head}\thead: {sent.words[word.head-1].text if word.head > 0 else "root"}\tdeprel: {word.deprel}' for sent in doc.sentences for word in sent.words], sep='\n')

In [ ]:
doc.sentences[0].print_dependencies()

Парсинг составляющих (для русского недоступен)

In [ ]:
doc_en = nlp_en('This is a sentence for parsing constituencies.')

for sentence in doc_en.sentences:
    print(sentence.constituency)

**Упражнения**

Сделайте токенизацию и морфосинтаксический парсинг для одного предложения на любом языке (доступность моделей для вашего языка можно посмотреть [здесь](https://stanfordnlp.github.io/stanza/available_models.html).

In [ ]:
# your code here